+ Sistem pemberi rekomendasi restoran adalah aplikasi yang merekomendasikan restoran sejenis kepada pelanggan sesuai dengan selera pelanggan.

In [1]:
# Import Library

import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load dataset
data = pd.read_csv("dataset/Restaurant/TripAdvisor_RestauarantRecommendation.csv")


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3062 entries, 0 to 3061
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              3062 non-null   object
 1   Street Address    3062 non-null   object
 2   Location          3062 non-null   object
 3   Type              3049 non-null   object
 4   Reviews           3062 non-null   object
 5   No of Reviews     3062 non-null   object
 6   Comments          2447 non-null   object
 7   Contact Number    3062 non-null   object
 8   Trip_advisor Url  3062 non-null   object
 9   Menu              3062 non-null   object
 10  Price_Range       3062 non-null   object
dtypes: object(11)
memory usage: 263.3+ KB


+ Mari kita lihat apakah datanya mengandung nilai nol atau tidak:

In [4]:
data.isnull().sum()

Name                  0
Street Address        0
Location              0
Type                 13
Reviews               0
No of Reviews         0
Comments            615
Contact Number        0
Trip_advisor Url      0
Menu                  0
Price_Range           0
dtype: int64

+ Saya akan memilih dua kolom dari kumpulan data untuk sisa tugas (Name, Type):

In [5]:
data = data[["Name", "Type"]]
print(data.head())

                            Name                                          Type
0  Betty Lou's Seafood and Grill   Seafood, Vegetarian Friendly, Vegan Options
1              Coach House Diner          Diner, American, Vegetarian Friendly
2               Table Talk Diner          American, Diner, Vegetarian Friendly
3                    Sixty Vines       American, Wine Bar, Vegetarian Friendly
4                   The Clam Bar                        American, Bar, Seafood


+ Jadi data memiliki beberapa nilai null di kolom Type. Saya akan menghapus baris yang berisi nilai nol sebelum melanjutkan:

In [6]:
data = data.dropna()

+ Jenis restoran merupakan fitur berharga dalam data untuk membangun sistem rekomendasi. 
+ Kolom jenis di sini mewakili kategori restoran. 
+ Misalnya, jika pelanggan menyukai restoran ramah vegetarian, dia hanya akan melihat rekomendasi jika restoran tersebut juga ramah vegetarian. 
+ Jadi saya akan menggunakan kolom Jenis sebagai fitur untuk merekomendasikan restoran serupa kepada pelanggan:

In [7]:
feature = data["Type"].tolist()
tfidf = text.TfidfVectorizer(input=feature, stop_words="english")
tfidf_matrix = tfidf.fit_transform(feature)
similarity = cosine_similarity(tfidf_matrix)

+ Sekarang saya akan menetapkan nama restoran sebagai indeks sehingga kita dapat menemukan restoran serupa dengan memberikan nama restoran sebagai input:



In [8]:
indices = pd.Series(data.index, index=data['Name']).drop_duplicates()

+ Write a function to recommend similar restaurants:

In [9]:
def restaurant_recommendation(name, similarity = similarity):
    index = indices[name]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    restaurantindices = [i[0] for i in similarity_scores]
    return data['Name'].iloc[restaurantindices]

In [10]:
# View recommendation

print(restaurant_recommendation("Market Grill"))

23                   The Lion's Share
154                        Houlihan's
518            Midgley's Public House
568                 Aspen Creek Grill
770              Pete's Sunset Grille
1190     Paul Martin's American Grill
1581                   Aviation Grill
1872                   Aviation Grill
2193                Crest Bar & Grill
2612    Tahoe Joe's Famous Steakhouse
Name: Name, dtype: object
